In [ ]:
workdir_ = '/glade/work/juliob/'
import sys
#######################################
# Leave this for now. But it should change to better
# method as here:
# import os
# module_a_dir = os.path.dirname(os.path.abspath(__file__))
# utils_path = os.path.join(module_a_dir, '..', 'Utils')
# sys.path.append(utils_path)
# print( f" a path added in {__name__} {utils_path} ")
########################################
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
#sys.path.append(workdir_ + 'PyRegridding/Regridder/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')

# Own local packages
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import utils as uti

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
importlib.reload(vAB)
importlib.reload(MkP)


In [ ]:
import matplotlib
print(matplotlib.__version__)
print(cartopy.__version__)

In [ ]:



"""
B = uti.MakeDict4Exp( exp='c64_005mm_ne30pg3_FMTHIST_NOmovmtn' ,
                     user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )
"""

B = uti.MakeDict4Exp( exp='c64_005mm_ne30pg3_FMTHIST_CTL' ,
                     user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

C = uti.MakeDict4Exp( exp='c64_005cac_ne30pg3_FMTHIST_CTL' ,
                     user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )
"""

B = uti.MakeDict4Exp( exp='c153_topfix_ne240pg3_FMTHIST_aicn_x01' ,
                     user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )

C = uti.MakeDict4Exp( exp='c153_topfix_ne240pg3_FMTHIST_L56_86km_aicn_x01' ,
                     user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )
"""

A = uti.MakeDict4Exp( exp='c153_ne30pg3_FMTHIST_x02'   , user='juliob', subd='regridded' , 
                     hsPat='cam.h0a' , ymdPat='*' ,verbose=True, open_dataset=True )



exp_C=C.exp
exp_B=B.exp
exp_Val='ERA5'

In [ ]:
########################
# Some massaging
########################


print( f'{A.exp} Years {len(A.X.time) /12. }')
print( f'{B.exp} Years {len(B.X.time) /12. }')
print( f'{C.exp} Years {len(C.X.time) /12. }')


##
#ps_c =Dc.PS.values
lon_c=C.X.lon.values
lat_c=C.X.lat.values
lev_c=C.X.lev.values
zlev_c=-7.0*np.log( lev_c/1_000. )

hyai_c=C.X.hyai.values
hybi_c=C.X.hybi.values
hyam_c=C.X.hyam.values
hybm_c=C.X.hybm.values

#ps_x =Dx.PS.values
lon_a=A.X.lon.values
lat_a=A.X.lat.values
lev_a=A.X.lev.values
zlev_a=-7.0*np.log( lev_a/1_000. )

lon_b=B.X.lon.values
lat_b=B.X.lat.values
lev_b=B.X.lev.values
zlev_b=-7.0*np.log( lev_b/1_000. )


In [ ]:
season='jja'
UUc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='U',return_time=True)
UUb_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='U',return_time=True)
UUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='U',return_time=True)

In [ ]:
UUc_sz=np.average( UUc_s , axis=2 )
UUb_sz=np.average( UUb_s , axis=2 )
UUa_sz=np.average( UUa_s , axis=2 )

In [ ]:
####
# Validation
####
Validation_on_control_mgrid = False

if (Validation_on_control_mgrid == True):
    PSc_s = Av.Seasonal( ds=Dc, season=season , fld='PS')
    mgrid={'ps':PSc_s, 'hyam':hyam_c, 'hybm':hybm_c, 'hyai':hyai_c, 'hybi':hybi_c, 'hgrid':'fv0.9x1.25' }
else:
    mgrid=False

Uval = Val.data(fld='U',season=season,mgrid=mgrid,zlev=True ) #, Years='2000' )
UUv_s , zlev_v,lat_v,lon_v = Uval['aa'], Uval['lev'], Uval['lat'], Uval['lon']

UUv_sz=np.average( UUv_s , axis=2 )

In [ ]:
fig = plt.figure(figsize=(24, 5))
clev=np.linspace( -60,140,num=21) 
dlev=np.linspace( -20,20,num=21) 
cmap='gist_ncar'
exp_A=A.exp

Axes1 = Pu.axes_def(n=1,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,UUa_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,UUa_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_A}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=3,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,UUc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,UUc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

Axes1 = Pu.axes_def(n=2,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_b,zlev_b,UUb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_b,zlev_b,UUb_sz ,levels=clev, colors='black')
ax1.set_title(f"Test <{exp_B}> {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


Axes1 = Pu.axes_def(n=4,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)


In [ ]:
#season='jul'
OUc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGAU',return_time=True)
OUb_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGAU',return_time=True)
OUa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGAU',return_time=True)

Oc_s,years_c,months_c = Av.Seasonal( ds=C.X, season=season , fld='OMEGA',return_time=True)
Ob_s,years_b,months_b = Av.Seasonal( ds=B.X, season=season , fld='OMEGA',return_time=True)
Oa_s,years_a,months_a = Av.Seasonal( ds=A.X, season=season , fld='OMEGA',return_time=True)

In [ ]:
OUpa_s = OUa_s - Oa_s*UUa_s
OUpb_s = OUb_s - Ob_s*UUb_s
OUpc_s = OUc_s - Oc_s*UUc_s


OUpa_sz=np.average( OUpa_s , axis=2 )
OUpb_sz=np.average( OUpb_s , axis=2 )
OUpc_sz=np.average( OUpc_s , axis=2 )



In [ ]:
fig = plt.figure(figsize=(20, 5))
clev=0.5*np.linspace( -0.1,0.1,num=21) 
dlev=21 #np.linspace( -20,20,num=21) 
cmap='gist_ncar'
exp_A=A.exp
fldoo="$\overline{u'\omega'}$"

n=1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_a,zlev_a,OUpa_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_a,zlev_a,OUpa_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_A}> {season.upper()} {years_a[0]}-{years_a[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_b,zlev_b,OUpb_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_b,zlev_b,OUpb_sz ,levels=clev, colors='black')
ax1.set_title(f"{fldoo} <{exp_B}> {season.upper()} {years_b[0]}-{years_b[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)

n=n+1
Axes1 = Pu.axes_def(n=n,nxplo=3,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_c,zlev_c,OUpc_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_c,zlev_c,OUpc_sz ,levels=clev, colors='black')
ax1.set_title(f"Control <{exp_C}> {season.upper()} {years_c[0]}-{years_c[-1]}" )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)



"""
Axes1 = Pu.axes_def(n=4,nxplo=4,nyplo=1 ) 
ax1 = fig.add_axes( Axes1 )
co1=ax1.contourf(lat_v,zlev_v, UUv_sz ,levels=clev, cmap=cmap )
co2=ax1.contour(lat_v,zlev_v, UUv_sz ,levels=clev, colors='black')
ax1.set_title(f"Validation <{exp_Val}> {season.upper()}  " )
ax1.set_ylim(0,82)
cb=plt.colorbar(co1)
"""

In [ ]:
C.X